## Projet Deep Learning II - DNN

In [ ]:
from models.DNN import DNN
from models.DBN import DBN
from models.RBM import RBM
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets

# Traitement de la donnée

In [ ]:
# similar to transforms.PILToTensor() but binarizes the image
from torchvision.utils import  _log_api_usage_once
from torchvision.transforms import functional as F

class PILToBinTensor:
    """Convert a PIL Image to a tensor of the same type - this does not scale values.

    This transform does not support torchscript.

    Converts a PIL Image (H x W x C) to a Tensor of shape (C x H x W).
    """

    def __init__(self) -> None:
        _log_api_usage_once(self)

    def __call__(self, pic):
        """
        .. note::

            A deep copy of the underlying array is performed.

        Args:
            pic (PIL Image): Image to be converted to tensor.

        Returns:
            Tensor: Converted image.
        """
        tensor = F.pil_to_tensor(pic)
        tensor[tensor <= 127] = 0
        tensor[tensor > 127] = 1
        return tensor

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}()"

In [ ]:
batch_size = 100

In [ ]:
# MNIST Dataset from yann.lecun.com
mnist_trainset = datasets.MNIST(root='./data/', train=True, transform=PILToBinTensor(), download=True)
mnist_testset = datasets.MNIST(root='./data/', train=False, transform=PILToBinTensor(), download=False)

#create data loader
max_mnist_size = 60000
mnist_trainset_reduced = torch.utils.data.random_split(mnist_trainset, [max_mnist_size, len(mnist_trainset)-max_mnist_size])[0] 
mnist_train_loader = torch.utils.data.DataLoader(mnist_trainset_reduced, batch_size=batch_size, shuffle=True,drop_last=True)

# download test dataset
max_mnist_size = 10000
mnist_testset_reduced = torch.utils.data.random_split(mnist_testset, [max_mnist_size, len(mnist_testset)-max_mnist_size])[0] 
mnist_test_loader = torch.utils.data.DataLoader(mnist_testset_reduced, batch_size=batch_size, shuffle=True,drop_last=True)

In [ ]:
n_rows = mnist_trainset_reduced.dataset.data.shape[1]
n_cols = mnist_trainset_reduced.dataset.data.shape[2]
n_channels = 1
n_pixels = n_rows*n_cols
classification_size = 10
img_shape = (n_rows, n_cols, n_channels)

In [ ]:
# unpack data and labels
#train data
train_data, train_labels,train_clear_label = [], [], []
for data, label in mnist_train_loader:
    train_data.append(data.reshape(-1,n_rows * n_cols * n_channels))
    train_labels.append(torch.nn.functional.one_hot(label, num_classes=classification_size))
    train_clear_label.append(label)
train_data = torch.cat(train_data).numpy()
train_labels = torch.cat(train_labels).numpy()
train_clear_label = torch.cat(train_clear_label).numpy()

#test data

# unpack data and labels
#test data
test_data, test_labels, test_clear_label = [], [], []
for data, label in mnist_test_loader:
    test_data.append(data.reshape(-1,n_rows * n_cols * n_channels))
    test_labels.append(torch.nn.functional.one_hot(label, num_classes=classification_size))
    test_clear_label.append(label)
test_data = torch.cat(test_data).numpy()
test_labels = torch.cat(test_labels).numpy()
test_clear_label = torch.cat(test_clear_label).numpy()

#### Configuration

In [ ]:
num_iterations_mnist = 200
num_iterations = 100
learning_rate = 0.1

#### Test DNN en variant le nombre de couches

In [ ]:
n_couches = [1,2,3,4,5,6,7]
dnn_pretrain_accuracy = []
dnn_random_accuracy = []
for i in n_couches :
    couches_dnn = [n_pixels]
    for _ in range(i):
        couches_dnn.append(200)
    dnn_pretrain = DNN()
    dnn_pretrain.init_DNN(couches_dnn, classification_size)
    dnn_pretrain.pretrain_DNN(num_iterations, learning_rate, batch_size, train_data)
    dnn_pretrain.retropropagation(train_data, train_labels, learning_rate, batch_size, num_iterations_mnist)
    dnn_pretrain_accuracy.append(dnn_pretrain.test_DNN(test_data,test_labels))

    dnn_random = DNN()
    dnn_random.init_DNN(couches_dnn, classification_size)
    dnn_random.retropropagation(train_data, train_labels, learning_rate, batch_size, num_iterations_mnist)
    dnn_random_accuracy.append(dnn_pretrain.test_DNN(test_data,test_labels))

In [ ]:
dnn_pretrain_error = 1 - np.array(dnn_pretrain_accuracy)
dnn_random_error = 1 - np.array(dnn_random_accuracy)
plt.plot(n_couches,dnn_pretrain_error,label = "DNN pré-entrainé")
plt.scatter(n_couches,dnn_pretrain_error)
plt.plot(n_couches,dnn_random_error,label = "DNN non pré-entrainé")
plt.scatter(n_couches,dnn_random_error)
plt.xlabel("nombre de couches")
plt.ylabel("taux d'erreur")
plt.legend()
plt.show()

#### Test DNN en variant le nombre de neurones par couche

In [ ]:
n_neurones = [100,300,500,700]
dnn_pretrain_accuracy = []
dnn_random_accuracy = []
for n_neurone in n_neurones :
    couches_dnn = [n_pixels,n_neurone,n_neurone]
    dnn_pretrain = DNN()
    dnn_pretrain.init_DNN(couches_dnn, classification_size)
    dnn_pretrain.pretrain_DNN(num_iterations, learning_rate, batch_size, train_data)
    dnn_pretrain.retropropagation(train_data, train_labels, learning_rate, batch_size, num_iterations_mnist)
    dnn_pretrain_accuracy.append(dnn_pretrain.test_DNN(test_data,test_labels))

    dnn_random = DNN()
    dnn_random.init_DNN(couches_dnn, classification_size)
    dnn_random.retropropagation(train_data, train_labels, learning_rate, batch_size, num_iterations_mnist)
    dnn_random_accuracy.append(dnn_pretrain.test_DNN(test_data,test_labels))

In [ ]:
dnn_pretrain_error = 1 - np.array(dnn_pretrain_accuracy)
dnn_random_error = 1 - np.array(dnn_random_accuracy)
plt.plot(n_neurones,dnn_pretrain_error,label = "DNN pré-entrainé")
plt.scatter(n_neurones,dnn_pretrain_error)
plt.plot(n_neurones,dnn_random_error,label = "DNN non pré-entrainé")
plt.scatter(n_neurones,dnn_random_error)
plt.xlabel("nombre de neuronnes")
plt.ylabel("taux d'erreur")
plt.legend()
plt.show()

#### Test DNN en variant le nombre de données d'apprentissage

In [ ]:
n_donnees_entrainement = [200, 1000, 3000, 7000, 10000, 30000, 60000]
dnn_pretrain_accuracy = []
dnn_random_accuracy = []
for n_entrainement in n_donnees_entrainement :
    couches_dnn = [n_pixels,n_neurone,n_neurone]
    dnn_pretrain = DNN()
    dnn_pretrain.init_DNN(couches_dnn, classification_size)
    dnn_pretrain.pretrain_DNN(num_iterations, learning_rate, batch_size, train_data[0:n_entrainement])
    dnn_pretrain.retropropagation(train_data[0:n_entrainement], train_labels[0:n_entrainement], learning_rate, batch_size, num_iterations_mnist)
    dnn_pretrain_accuracy.append(dnn_pretrain.test_DNN(test_data,test_labels))

    dnn_random = DNN()
    dnn_random.init_DNN(couches_dnn, classification_size)
    dnn_random.retropropagation(train_data[0:n_entrainement], train_labels[0:n_entrainement], learning_rate, batch_size, num_iterations_mnist)
    dnn_random_accuracy.append(dnn_pretrain.test_DNN(test_data,test_labels))

In [ ]:
dnn_pretrain_error = 1 - np.array(dnn_pretrain_accuracy)
dnn_random_error = 1 - np.array(dnn_random_accuracy)
plt.plot(n_donnees_entrainement,dnn_pretrain_error,label = "DNN pré-entrainé")
plt.scatter(n_donnees_entrainement,dnn_pretrain_error)
plt.plot(n_donnees_entrainement,dnn_random_error,label = "DNN non pré-entrainé")
plt.scatter(n_donnees_entrainement,dnn_random_error)
plt.xlabel("nombre de données d'entraînement")
plt.ylabel("taux d'erreur")
plt.legend()
plt.show()